In [6]:
import sqlite3
import csv
import os
import time
import logging

# ——— Setup logging ———
logging.basicConfig(
    filename="logs/ingestion_db.log",
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# ——— Config ———
DATA_DIR = "data"
DB_PATH  = "inventory.db"
BATCH_SZ = 10_000    # commit every 10k rows

def ingest_csv(file_path, conn):
    """
    Stream a CSV into SQLite with pure sqlite3:
      - creates (or replaces) a table named after the file (minus “.csv”)
      - reads rows in batches of BATCH_SZ
      - uses executemany() per batch (never > 1 row of placeholders)
    """
    table_name = os.path.splitext(os.path.basename(file_path))[0]
    cur = conn.cursor()

    # open CSV & read header
    with open(file_path, newline="", encoding="utf-8") as f:
        reader = csv.reader(f)
        headers = next(reader)
        cols_def = ", ".join(f'"{h}" TEXT' for h in headers)

        # (re)create table
        cur.execute(f'DROP TABLE IF EXISTS "{table_name}"')
        cur.execute(f'CREATE TABLE "{table_name}" ({cols_def})')

        # prepare INSERT
        placeholder = ", ".join("?" for _ in headers)
        sql_insert  = f'INSERT INTO "{table_name}" VALUES ({placeholder})'

        # stream rows in batches
        batch = []
        for row in reader:
            batch.append(tuple(row))
            if len(batch) >= BATCH_SZ:
                cur.executemany(sql_insert, batch)
                conn.commit()
                batch.clear()

        # final partial batch
        if batch:
            cur.executemany(sql_insert, batch)
            conn.commit()

    logging.info(f"Finished ingesting {table_name} ({file_path})")

def load_all_csvs():
    start = time.time()
    conn = sqlite3.connect(DB_PATH)
    try:
        for fname in os.listdir(DATA_DIR):
            if not fname.lower().endswith(".csv"):
                continue
            path = os.path.join(DATA_DIR, fname)
            logging.info(f"Starting {fname}")
            ingest_csv(path, conn)
    finally:
        conn.close()

    mins = (time.time() - start) / 60
    logging.info(f"All files ingested in {mins:.2f} minutes")

if __name__ == "__main__":
    load_all_csvs()
